## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [366]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [367]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [368]:
# Use the describe function to gather some basic statistics

df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00
mean,54837.87,5470190.03,11.36,2022.99,136.27,99.45,5.00,570.26,2.98
std,25487.44,2599806.93,7.02,0.08,183.87,133.16,5.33,18795.52,2.44
min,10033.00,1000886.00,1.00,2022.00,0.01,0.01,0.00,0.00,0.00
25%,33593.00,3196372.00,6.00,2023.00,20.80,14.84,1.45,32.00,1.00
50%,53305.00,5496966.00,11.00,2023.00,68.31,49.89,3.24,68.00,3.00
75%,78498.00,7733869.00,17.00,2023.00,173.16,125.57,6.89,170.00,5.00
max,99984.00,9998480.00,52.00,2023.00,1396.23,846.27,46.43,3958244.00,9.00


In [369]:
# Use this space to do any additional research
# and familiarize yourself with the data.



In [370]:
# What three item categories had the most entries?

category_count = df.value_counts(subset = 'category')
category_count.head(3)



category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [371]:
# For the category with the most entries,
# which subcategory had the most entries?
subcategory_count = df[df['category'] == 'consumables']['subcategory'].value_counts()
subcategory_count.head(1)



subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [372]:
# Which five clients had the most entries in the data?
top_clients = df.value_counts(subset='client_id')
top_clients.head(5)


client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [373]:

# Store the client ids of those top 5 clients in a list.
top_client_list = top_clients.head(5).index.tolist()
top_client_list

[33615, 66037, 46820, 38378, 24741]

In [374]:
# How many total units (the qty column) did the
# client with the most entries order order?
top_client = df[df['client_id'] == 33615]
top_client_qty = top_client['qty'].sum()
top_client_qty


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [375]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['subtotal'] = df['unit_price'] * df['qty']
subtotal_df = df[['unit_price', 'qty', 'subtotal']]
subtotal_df

,unit_price,qty,subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28
3,36.42,29,1056.18
4,195.10,20,3902.00
...,...,...,...
54634,83.13,33,2743.29
54635,206.59,47,9709.73
54636,65.66,475,31188.50
54637,1.48,112,165.76


In [376]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
df['total_weight'] = df['qty'] * df['unit_weight']
df['shipping_price'] = df['total_weight'].apply(lambda x: x * 7.0 if x > 50 else x * 10.0)
shipping_price_df = df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']]
shipping_price_df




,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64
3,36.42,1.23,29,35.67,356.70
4,195.10,46.43,20,928.60,6500.20
...,...,...,...,...,...
54634,83.13,2.25,33,74.25,519.75
54635,206.59,11.70,47,549.90,3849.30
54636,65.66,4.16,475,1976.00,13832.00
54637,1.48,18.04,112,2020.48,14143.36


In [377]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
df['sales_tax'] = (df['subtotal'] + df['shipping_price']) * 0.0925
df['total_price'] = (df['subtotal'] + df['shipping_price'] + df['sales_tax']).round(2)
total_price_df = df[['subtotal', 'shipping_price', 'total_price']]
total_price_df



,subtotal,shipping_price,total_price
0,115164.00,5512.50,131839.08
1,523.95,312.90,914.26
2,527.28,458.64,1077.12
3,1056.18,356.70,1543.57
4,3902.00,6500.20,11364.40
...,...,...,...
54634,2743.29,519.75,3564.87
54635,9709.73,3849.30,14813.24
54636,31188.50,13832.00,49184.90
54637,165.76,14143.36,15632.71


In [378]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
df['total_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,subtotal,total_weight,shipping_price,sales_tax,total_price,total_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,11162.58,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,77.41,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,91.20,1077.12,765.18
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,130.69,1543.57,1077.35
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,962.20,11364.40,8663.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,51.60,2.25,33,8,2743.29,74.25,519.75,301.83,3564.87,2222.55
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,175.46,11.70,47,0,9709.73,549.90,3849.30,1254.21,14813.24,12095.92
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,57.31,4.16,475,6,31188.50,1976.00,13832.00,4164.40,49184.90,41054.25
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,1.22,18.04,112,0,165.76,2020.48,14143.36,1323.59,15632.71,14280.00


In [379]:
# Create a column for the profit of
# each line using line cost and line price
df['profit'] = df['total_price'] - df['total_cost']
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_weight,qty,line_number,subtotal,total_weight,shipping_price,sales_tax,total_price,total_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,7.50,105,1,115164.00,787.50,5512.50,11162.58,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,1.49,21,0,523.95,31.29,312.90,77.41,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,1.68,39,6,527.28,65.52,458.64,91.20,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,1.23,29,3,1056.18,35.67,356.70,130.69,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,46.43,20,1,3902.00,928.60,6500.20,962.20,11364.40,8663.60,2700.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,2.25,33,8,2743.29,74.25,519.75,301.83,3564.87,2222.55,1342.32
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,11.70,47,0,9709.73,549.90,3849.30,1254.21,14813.24,12095.92,2717.32
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,4.16,475,6,31188.50,1976.00,13832.00,4164.40,49184.90,41054.25,8130.65
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,18.04,112,0,165.76,2020.48,14143.36,1323.59,15632.71,14280.00,1352.71


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [380]:
# Check your work using the totals above
order_2742071 = round(sum(df.loc[df['order_id'] == 2742071, 'total_price']), 2)
print(order_2742071)
order_2173913 = round(sum(df.loc[df['order_id'] == 2173913, 'total_price']), 2)
print(order_2173913)
order_6128929 = round(sum(df.loc[df['order_id'] == 6128929, 'total_price']), 2)
print(order_6128929)



152811.89
162388.71
923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [381]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
#top_client_spend = [spend for client_id, spend in zip(df['client_id'], df['total_price']) if client_id in top_client_list]
pd.options.display.float_format = '{:.2f}'.format

client_spend = df.groupby(['client_id', 'qty'])['total_price'].sum()
client_spend = client_spend.reset_index('client_id')
client_spend_sorted = client_spend.sort_values(by="total_price", ascending=False)
client_spend_sorted.head(5)




,client_id,total_price
qty,,
3958244,78965,641262543.02
637318,85931,235958911.49
977121,86721,176074223.98
219645,38473,165739466.02
212123,62227,97867968.06


In [382]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,


top_qty=[]
top_shipping_price = []
top_total_price = []
top_profit = []

for index, row in df.iterrows():
    client_id = row['client_id']
    if client_id in top_client_list:
        top_qty.append(row['qty'])
        top_shipping_price.append(row['shipping_price'])
        top_total_price.append(row['total_price'])
        top_profit.append(row['profit'])
        
sums_per_client={}

for client_id in top_client_list:
    indices = df[df['client_id'] == client_id].index
    sum_top_qty_client = sum(df.loc[indices,'qty'])
    sum_top_shipping_price_client = sum(df.loc[indices, 'shipping_price'])
    sum_top_total_price_client = sum(df.loc[indices, 'total_price'])
    sum_top_profit_client = sum(df.loc[indices, 'profit'])
    sums_per_client[client_id] = {'qty' : sum_top_qty_client, 'shipping_price' : sum_top_shipping_price_client, 'total_price' : sum_top_total_price_client, 'profit': sum_top_profit_client}

top_client_sums = pd.DataFrame(sums_per_client).T

top_client_sums = top_client_sums.rename_axis("client_id")

top_client_sums



,qty,shipping_price,total_price,profit
client_id,,,,
33615,64313.00,1828984.89,8377308.52,2201994.61
66037,43018.00,1395151.85,10259514.79,3255031.81
46820,75768.00,1601448.84,9743794.36,2736602.72
38378,73667.00,3429455.40,12906550.87,3271829.89
24741,239862.00,5126448.37,82268892.02,36579992.31


In [383]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

top_client_sums = top_client_sums.rename(columns={'qty' : 'Quantity' , 'shipping_price' : 'Shipping Price' , 'total_price' :  'Total Price' , 'profit' : 'Profit'})
top_client_sums = top_client_sums.rename_axis('Client ID')

top_client_sums_mil = top_client_sums.div(1000000).round(3)
    

top_client_sums_mil




,Quantity,Shipping Price,Total Price,Profit
Client ID,,,,
33615,0.06,1.83,8.38,2.20
66037,0.04,1.40,10.26,3.26
46820,0.08,1.60,9.74,2.74
38378,0.07,3.43,12.91,3.27
24741,0.24,5.13,82.27,36.58


In [385]:
# Sort the updated data by "Total Profit" form highest to lowest

top_client_sums_mil_sort = top_client_sums_mil.sort_values(by='Profit', ascending=False)

top_client_sums_mil_sort


,Quantity,Shipping Price,Total Price,Profit
Client ID,,,,
24741,0.24,5.13,82.27,36.58
38378,0.07,3.43,12.91,3.27
66037,0.04,1.40,10.26,3.26
46820,0.08,1.60,9.74,2.74
33615,0.06,1.83,8.38,2.20
